In [6]:
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lyricsgenius
import requests
import re
import lyricsgenius
import csv
from bs4 import BeautifulSoup

"""
The Year-end Wiki scrapper, was taken from here:
https://github.com/awcooper/spotify_billboard_playlist_generator/blob/master/playlist_generators/hot_100_scraper_v1.py
"""
def get_billboard_song_titles_for_year(year):
    """
    :return: List of billboard songs and artists in a tuple '(song, artist)'
    """
    billboard_page = "https://www.wikizero.com/en/Billboard_Year-End_Hot_100_singles_of_"
    page = requests.get(billboard_page + str(year))
    soup = BeautifulSoup(page.content, 'html.parser')
    doc = soup.find("table", {"class": "wikitable"})
    year_data = []
    for row in doc.find_all(["tr"])[1:]:
        # The th is required because ~2000+ uses that format instead
        row_data = [cell.text.strip() for cell in row.findAll(["td", "th"])]
        if len(row_data) != 3:
            print("Error Processing Row: ", row)
        else:
            year_data.append(tuple(row_data))
    return year_data

"""Gets the main artist"""
def get_main_artist(content):
    """:return: main artist"""
    for split_token in [" & ", " \\ ", " feat ", " featuring ", " and "]:
        content = content.partition(split_token)[0]
    return content

"""Takes lyrics and removes annotations and empty lines"""
def clean_lyrics(lyrics):
    """:return:clean lyrics"""
    #Remove annotations
    lyrics =  re.sub(r'[\(\[].*?[\)\]]', '',lyrics)
    
    #Remove empty lines and commas
    lyrics = [line.replace(',','') for line in lyrics.split('\n') if line.strip() != '']
    lyrics = "\n".join(lyrics)
    
    return lyrics

"""Writes a year's lyrics into a .csv file"""
def write_lyrics(year):
    token = "MY_bmSPzzEQDrlRLp9xS77g-vQqtCtimk8Yw4toCkyduF7eUbrPI-IPxXATgAESi"
    genius = lyricsgenius.Genius(token)
    
    with open('lyrics/'+year+'.csv', 'a', newline="") as csvfile:
        filewriter = csv.writer(
            csvfile,
            delimiter=',',
            quotechar='|',
            quoting=csv.QUOTE_MINIMAL
        )
        
        filewriter.writerow(['Rank', 'Song', 'Artist', 'Year','Lyrics'])
        year_songs = get_billboard_song_titles_for_year(year)
        for song in year_songs:
            song_rank = song[0]
            song_name = song[1]
            song_artist = get_main_artist(song[2])
            song_lyrics = clean_lyrics(genius.search_song(song_name, song_artist).lyrics)
            filewriter.writerow([song_rank, song_name.replace(',',''), song_artist.replace(',',''), year,song_lyrics.encode('utf-8')])
            
"""Combine .csv files"""
def combine_files(first_year,last_year):
    fout = open("lyrics/bh100.csv","a")
    for number in range(first_year,last_year+1):
        f = open("lyrics/20"+str(number)+".csv")
        #sktpping the header
        if number != first_year:
            next(f)
        for line in f:
            fout.write(line)
        f.close()
    fout.close()
    

def main():
    # The splitting via years was made in case of request timeouts
    years= ["2016","2017","2018"]
    for year in years:
            write_lyrics(year)
    combine_files(16,18)
    
main()


  Using cached https://files.pythonhosted.org/packages/9e/d4/10f46e5cfac773e22707237bfcd51bbffeaf0a576b0a847ec7ab15bd7ace/beautifulsoup4-4.6.0-py3-none-any.whl


No metadata found in c:\users\sdvii\miniconda3\lib\site-packages
Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: 'c:\\users\\sdvii\\miniconda3\\lib\\site-packages\\beautifulsoup4-4.7.1.dist-info\\METADATA'

Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: 'c:\\users\\sdvii\\miniconda3\\lib\\site-packages\\beautifulsoup4-4.7.1.dist-info\\METADATA'



TypeError: a bytes-like object is required, not 'str'